# ETL Project - Gun Violence

In [1]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password
import requests
from bs4 import BeautifulSoup
URL = "https://en.wikipedia.org/wiki/Gun_laws_in_the_United_States_by_state"

# Mass Shooting, Accidental Deaths and Accidental Injuries - Extract and Transform

In [2]:
df_1 = pd.read_csv("Resources/General_Accidental_Death.csv")
df_2 = pd.read_csv("Resources/General_Accidental_Injuries.csv")
df_3 = pd.read_csv("Resources/Mass_shootings.csv")

In [3]:
df_1.columns = ['ID', 'Date', 'State', 'City', 'Address', 'Killed', 'Injured', 'Op']
df_2.columns = ['ID', 'Date', 'State', 'City', 'Address', 'Killed', 'Injured', 'Op']
df_3.columns = ['ID', 'Date', 'State', 'City', 'Address', 'Killed', 'Injured', 'Op']

del df_1['Op']
del df_2['Op']
del df_3['Op']

df_1['Type'] = "Accident"
df_1['Type'] = "Accident"
df_3['Type'] = "Mass Shooting"

In [4]:
df = pd.DataFrame(columns = ['ID', 'Date', 'State', 'City', 'Address', 'Killed', 'Injured'])

In [5]:
df = df.append(df_1)
df = df.append(df_2)
df = df.append(df_3)

In [6]:

df.head()

,ID,Date,State,City,Address,Killed,Injured,Type
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0,Accident
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0,Accident
2,1748086,26-Jul-20,Ohio,Grand Rapids,17000 block of Wapakoneta Rd,1,0,Accident
3,1746886,26-Jul-20,California,Los Angeles,Hollywood Blvd and N Highland Ave,1,1,Accident
4,1746495,25-Jul-20,Florida,Largo,71000 Ulmerton Rd,1,0,Accident


In [7]:
# Incidents
df.head()

,ID,Date,State,City,Address,Killed,Injured,Type
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0,Accident
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0,Accident
2,1748086,26-Jul-20,Ohio,Grand Rapids,17000 block of Wapakoneta Rd,1,0,Accident
3,1746886,26-Jul-20,California,Los Angeles,Hollywood Blvd and N Highland Ave,1,1,Accident
4,1746495,25-Jul-20,Florida,Largo,71000 Ulmerton Rd,1,0,Accident


And the Cleaning begins...

In [8]:
dropped_df=df[df.ID != '#######']
dropped_df

,ID,Date,State,City,Address,Killed,Injured,Type
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0,Accident
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0,Accident
2,1748086,26-Jul-20,Ohio,Grand Rapids,17000 block of Wapakoneta Rd,1,0,Accident
3,1746886,26-Jul-20,California,Los Angeles,Hollywood Blvd and N Highland Ave,1,1,Accident
4,1746495,25-Jul-20,Florida,Largo,71000 Ulmerton Rd,1,0,Accident
...,...,...,...,...,...,...,...,...
495,1475208,9-Aug-19,Illinois,Chicago,7300 block of S Artesian Ave,0,4,Mass Shooting
496,1472641,7-Aug-19,Missouri,Saint Louis,4900 block of Plover Ave,3,1,Mass Shooting
497,1469344,5-Aug-19,New York,Brooklyn,216 Buffalo Ave,0,4,Mass Shooting
498,1527721,4-Aug-19,Louisiana,Lake Charles,110 Pryce St,0,4,Mass Shooting


In [9]:
dropped_df[dropped_df.ID == '#######']

,ID,Date,State,City,Address,Killed,Injured,Type


In [10]:
dropped_df.duplicated(subset=['ID'])

0      False
1      False
2      False
3      False
4      False
       ...  
495    False
496    False
497    False
498    False
499    False
Length: 1500, dtype: bool

In [11]:
dropped_df['ID'].nunique()

1475

In [12]:
dropped_df[dropped_df.duplicated(subset=['ID']) == True]

,ID,Date,State,City,Address,Killed,Injured,Type
18,1740578,20-Jul-20,Texas,Arlington,4100 block of Maple Springs Dr,1,1,NaN
33,1738740,18-Jul-20,South Carolina,North Charleston,4755 Saul White Blvd,1,2,NaN
95,1726663,5-Jul-20,Georgia,Atlanta,1192 Pryor Rd,1,2,NaN
112,1723897,3-Jul-20,Alabama,Birmingham (Hoover),2000 Riverchase Galleria,1,3,NaN
134,1718645,26-Jun-20,West Virginia,Cottageville,NaN,1,1,NaN
157,1711272,20-Jun-20,Michigan,Saginaw,Livingston Dr and Tausend St,1,3,NaN
175,1706994,16-Jun-20,Iowa,Fort Dodge,900 block of 10th Ave SW,2,2,NaN
176,1707253,16-Jun-20,Illinois,Chicago,7600 block of N Marshfield Ave,1,1,NaN
355,1670035,1-May-20,Alabama,Pinson,9100 block of Cedar Mountain Rd,1,1,NaN
417,1656735,10-Apr-20,Florida,Palm Beach Gardens (Riviera Beach),1000 Block of W 26th St,1,1,NaN


In [13]:
clean_dropped_df=dropped_df.drop_duplicates(subset=['ID'])
clean_dropped_df.head()

,ID,Date,State,City,Address,Killed,Injured,Type
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0,Accident
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0,Accident
2,1748086,26-Jul-20,Ohio,Grand Rapids,17000 block of Wapakoneta Rd,1,0,Accident
3,1746886,26-Jul-20,California,Los Angeles,Hollywood Blvd and N Highland Ave,1,1,Accident
4,1746495,25-Jul-20,Florida,Largo,71000 Ulmerton Rd,1,0,Accident


In [14]:
clean_violence_df=clean_dropped_df.drop(['Type'],axis=1)
clean_violence_df.head()

,ID,Date,State,City,Address,Killed,Injured
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0
2,1748086,26-Jul-20,Ohio,Grand Rapids,17000 block of Wapakoneta Rd,1,0
3,1746886,26-Jul-20,California,Los Angeles,Hollywood Blvd and N Highland Ave,1,1
4,1746495,25-Jul-20,Florida,Largo,71000 Ulmerton Rd,1,0


In [15]:
mapping =clean_dropped_df[['State', 'City']].drop_duplicates()

del df['State']

# Mapping Tables

In [16]:
type_df=clean_dropped_df.drop(['Date','City','State','Address','Killed','Injured'],axis=1)
type_df.head()

,ID,Type
0,1747443,Accident
1,1747425,Accident
2,1748086,Accident
3,1746886,Accident
4,1746495,Accident


In [17]:
type_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1475 non-null   object
 1   Type    985 non-null    object
dtypes: object(2)
memory usage: 34.6+ KB


In [18]:
type_df.fillna('Unknown',inplace=True)

type_df.head()

,ID,Type
0,1747443,Accident
1,1747425,Accident
2,1748086,Accident
3,1746886,Accident
4,1746495,Accident


In [19]:
type_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1475 non-null   object
 1   Type    1475 non-null   object
dtypes: object(2)
memory usage: 34.6+ KB


In [20]:
# city_mapping
mapping.head()

,State,City
0,Arizona,Tucson
1,Tennessee,Nashville
2,Ohio,Grand Rapids
3,California,Los Angeles
4,Florida,Largo


In [21]:
new_map_df=pd.DataFrame(mapping)

In [22]:
new_map_df.duplicated(subset=['City'])

0      False
1      False
2      False
3      False
4      False
       ...  
470    False
475    False
477    False
480    False
491     True
Length: 745, dtype: bool

In [23]:
new_map_df['City'].nunique()

696

In [24]:
new_map_df[new_map_df.duplicated(subset=['City']) == True]

,State,City
154,Tennessee,Jackson
237,Georgia,Lexington
280,Georgia,Columbus
334,South Carolina,Springfield
343,Virginia,Waynesboro
355,New York,Frankfort
396,Missouri,Columbia
413,Georgia,Decatur
441,Michigan,Grand Rapids
456,Pennsylvania,Gordon


In [25]:
clean_map_df=new_map_df.drop_duplicates(subset=['City'])
clean_map_df

,State,City
0,Arizona,Tucson
1,Tennessee,Nashville
2,Ohio,Grand Rapids
3,California,Los Angeles
4,Florida,Largo
...,...,...
467,Texas,Odessa
470,Maryland,Frederick
475,New Mexico,Hobbs
477,Maryland,Temple Hills (Camp Springs)


In [26]:
clean_map_df['City'].nunique()

696

In [27]:
clean_map_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 0 to 480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   696 non-null    object
 1   City    696 non-null    object
dtypes: object(2)
memory usage: 16.3+ KB


# Web Scrape of Wikipedia on Gun Laws - Extract and Transform

In [28]:
states = pd.read_html(URL)
states = states[1:-3]
states

[                                     Subject/Law Long guns Handguns  \
 0             State permit required to purchase?        No       No   
 1                          Firearm registration?        No       No   
 2                        Owner license required?        No       No   
 3           Permit required for concealed carry?       NaN      Yes   
 4                Permit required for open carry?        No       No   
 5        State preemption of local restrictions?       Yes      Yes   
 6                            Assault weapon law?        No       No   
 7                        NFA weapons restricted?        No       No   
 8  Background checks required for private sales?        No       No   
 
    Relevant statutes                                              Notes  
 0                NaN                                                NaN  
 1                NaN                                                NaN  
 2                NaN                                

In [29]:
res = requests.get(URL)
soup = BeautifulSoup(res.content, 'html.parser')
headings = soup.find_all('span', class_='mw-headline')

temp = []
for i in headings:
    temp.append(i.get("id"))
state_names = temp[1:-4]
state_names

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District_of_Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming',
 'US_Territories',
 'American_Samoa',
 'Guam',
 'Northern_Mariana_Islands',
 'Puerto_Rico']

In [30]:
mapping = {}
for i in range(len(state_names)):
    mapping[ state_names[i] ] = states[i]

In [31]:
# a = [
#     ["California", "yes", "no"],
#     [4, 5, 6]
# ]
# pd.DataFrame(a, columns=["State Name", "Ques 1", "c"])

# State permit required to purchase?
ques=["State permit required to purchase?", "Firearm registration?", "Owner license required?", "Background checks required for private sales?"]

# Traverse the list
master_arr = []
for state in mapping:
    temp = [state]

    # print (state)
    df = mapping[state]
    try:
        df['Subject/Law'] = df['Subject/Law'].str.strip()
        df = df.set_index("Subject/Law")
    except:
        df['Subject/law'] = df['Subject/law'].str.strip()
        df = df.set_index("Subject/law")
#     df.at( index, row )
    for q in ques:
        try:
            temp.append( df.at[q, 'Long guns'] )
        except Exception as e:
            # print (e, state, q)
            try:
                temp.append( df.at[q, 'Long Guns'] )
            except Exception as e:
                # print (e, state, q)
                temp.append(float('nan'))

    master_arr.append(temp)

laws_df = pd.DataFrame(master_arr, columns=["State"] + ques)

In [32]:
laws_df.head()

,State,State permit required to purchase?,Firearm registration?,Owner license required?,Background checks required for private sales?
0,Alabama,No,No,No,No
1,Alaska,No,No,No,No
2,Arizona,No,NaN,No,No
3,Arkansas,No,No,No,No
4,California,Partial,Yes,No,Yes


In [33]:
#To rename the States so they would match other data frames
new_laws_df = laws_df.replace({'State': {'District_of_Columbia': 'District of Columbia', 'New_Hampshire': 'New Hampshire', 'New_Jersey': 'New Jersey', 
                           'New_Mexico': 'New Mexico', 'New_York': 'New York', 'North_Carolina': 'North Carolina', 
                           'North_Dakota': 'North Dakota', 'Rhode_Island': 'Rhode Island', 'West_Virginia': 'West Virginia',
                           'South_Carolina': 'South Carolina', 'South_Dakota': 'South Dakota', 'Puerto_Rico': 'Puerto Rico'}})
new_laws_df.head()

,State,State permit required to purchase?,Firearm registration?,Owner license required?,Background checks required for private sales?
0,Alabama,No,No,No,No
1,Alaska,No,No,No,No
2,Arizona,No,NaN,No,No
3,Arkansas,No,No,No,No
4,California,Partial,Yes,No,Yes


In [34]:
#To drop Specific external states that were not necessary for the necessary data
drop_laws_df = new_laws_df.drop([55, 54, 53, 52, 51])
drop_laws_df.head()

,State,State permit required to purchase?,Firearm registration?,Owner license required?,Background checks required for private sales?
0,Alabama,No,No,No,No
1,Alaska,No,No,No,No
2,Arizona,No,NaN,No,No
3,Arkansas,No,No,No,No
4,California,Partial,Yes,No,Yes


In [35]:
clean_laws_df = drop_laws_df.rename(columns = {'State permit required to purchase?':'State_permit_required_to_purchase','Firearm registration?':'Firearm_registration','Owner license required?':'Owner_license_required','Background checks required for private sales?':'Background_checks_required_for_private_sales'})
clean_laws_df.head()



,State,State_permit_required_to_purchase,Firearm_registration,Owner_license_required,Background_checks_required_for_private_sales
0,Alabama,No,No,No,No
1,Alaska,No,No,No,No
2,Arizona,No,NaN,No,No
3,Arkansas,No,No,No,No
4,California,Partial,Yes,No,Yes


Final Clean before Loading

In [36]:
clean_violence_df.columns= clean_violence_df.columns.str.lower()
clean_violence_df.head(2)

,id,date,state,city,address,killed,injured
0,1747443,27-Jul-20,Arizona,Tucson,25 W Calle Concordia,1,0
1,1747425,26-Jul-20,Tennessee,Nashville,500 block of Charles E Davis Blvd,1,0


In [37]:
clean_map_df.columns= new_map_df.columns.str.lower()
clean_map_df.head()

,state,city
0,Arizona,Tucson
1,Tennessee,Nashville
2,Ohio,Grand Rapids
3,California,Los Angeles
4,Florida,Largo


In [38]:
type_df.columns= type_df.columns.str.lower()
type_df.head(2)

,id,type
0,1747443,Accident
1,1747425,Accident


In [39]:
clean_laws_df.columns= clean_laws_df.columns.str.lower()
clean_laws_df.head(2)

,state,state_permit_required_to_purchase,firearm_registration,owner_license_required,background_checks_required_for_private_sales
0,Alabama,No,No,No,No
1,Alaska,No,No,No,No


# Load into PostgreSQL DB

In [40]:
rds_connection_string = f'{username}:{password}@localhost:5432/gun_violence'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [41]:
clean_laws_df.to_sql(name='Gun_Laws', con=engine, if_exists='append', index=False)
type_df.to_sql(name='Type', con=engine, if_exists='append', index=False)
clean_map_df.to_sql(name='State', con=engine, if_exists='append', index=False)
clean_violence_df.to_sql(name='Violence', con=engine, if_exists='append', index=False)

In [42]:
engine.table_names()

['Gun_Laws', 'State', 'Type', 'Violence']